使用 torch.nn 包来构建神经网络

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_feat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_feat_features(self, x):
        size = x.size()[1:] # 所有维度，除了batch
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [2]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
parms = list(net.parameters())
print(len(parms))
print(parms[0].size())

10
torch.Size([6, 1, 5, 5])


In [6]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.1000,  0.0555,  0.0314,  0.0785,  0.0349,  0.0631,  0.0292, -0.0389,
         -0.1042, -0.0455]], grad_fn=<AddmmBackward0>)


In [7]:
net.zero_grad()
out.backward(torch.randn(1,10))

torch.nn 只支持小批量输入。整个 torch.nn 不支持单个样本

In [8]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.5212, grad_fn=<MseLossBackward0>)


In [11]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0066,  0.0066,  0.0078, -0.0044,  0.0067, -0.0069])


In [12]:
import  torch.optim  as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()